In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mercor-ai-detection/sample_submission.csv
/kaggle/input/mercor-ai-detection/train.csv
/kaggle/input/mercor-ai-detection/test.csv


In [2]:
# ===============================
# 🧹 STEP 1: Load and preprocess dataset for AI-text detection
# ===============================
import pandas as pd

# Adjust these paths as needed for your Kaggle environment
train_path = "/kaggle/input/mercor-ai-detection/train.csv"
test_path = "/kaggle/input/mercor-ai-detection/test.csv"

# Load data
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

print("Before merging:")
print(train.head())

# Combine topic and answer into a single text column
train["text"] = train["topic"].astype(str) + " " + train["answer"].astype(str)
test["text"] = test["topic"].astype(str) + " " + test["answer"].astype(str)

# Keep only necessary columns
train = train[["text", "is_cheating"]]
test = test[["id", "text"]]

print("After merging:")
print(train.head())

Before merging:
                                id  \
0    form_r_d5TpupthGXvfy58LDIbkqp   
1  form_r_AAABmK2rZBBbtfdHxzhBY4C4   
2    form_r_Cz6IJIWUj1B7pdyO6zPhGF   
3  form_r_AAABmKyCfdO6NLv8eqBPCYD_   
4    form_r_WS8I1NJpwvIcFL3Xv0Qhuc   

                                               topic  \
0  A girl wakes from a dream and she is not sure ...   
1  A journalistic review piece about the top 6 ai...   
2  The influence of fictional universities in cam...   
3                           Why do girls love horses   
4  Every year, a remote mountain town elects a ne...   

                                              answer  is_cheating  
0  My eyes flew open, and the air around me feels...            1  
1  Robot Butlers in the year of 2025. What are th...            0  
2  In recent years, apparel featuring the names a...            1  
3  The moment before I hit the dirt, I thought we...            0  
4  In the valley of Eldermist, were the mountains...            1  
After merg

In [3]:
print(train["is_cheating"].value_counts())

is_cheating
1    147
0    122
Name: count, dtype: int64


In [4]:
def random_split(df, train_frac=0.8):
    """
    Randomly splits a DataFrame into training and validation sets.
    
    Args:
        df (pd.DataFrame): The input dataset.
        train_frac (float): Fraction of data to use for training (default 0.8).
        
    Returns:
        train_df (pd.DataFrame), validation_df (pd.DataFrame)
    """
    # Shuffle the DataFrame
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)

    # Compute split index
    train_end = int(len(df) * train_frac)

    # Split into training and validation sets
    train_df = df[:train_end]
    validation_df = df[train_end:]

    return train_df, validation_df


# Example usage:
train_df, validation_df = random_split(train, train_frac=0.8)

# Optionally save to disk
train_df.to_csv("train.csv", index=False)
validation_df.to_csv("validation.csv", index=False)
test.to_csv("test.csv" ,index=False)

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(validation_df)}")


Training samples: 215
Validation samples: 54


In [5]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


In [6]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import tiktoken


class AIDetectionDataset(Dataset):
    def __init__(self, csv_file, tokenizer=None, max_length=1024, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)

        # Initialize tokenizer (use GPT-2 encoding if none provided)
        self.tokenizer = tokenizer or tiktoken.get_encoding("gpt2")

        # Pre-tokenize all texts
        self.encoded_texts = [
            self.tokenizer.encode(str(text)) for text in self.data["text"]
        ]

        # Determine or set max_length
        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length

        # Truncate and pad sequences
        self.encoded_texts = [
            (encoded[:self.max_length] + [pad_token_id] * max(0, self.max_length - len(encoded)))
            for encoded in self.encoded_texts
        ]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["is_cheating"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        """Find the maximum tokenized sequence length."""
        return max(len(encoded) for encoded in self.encoded_texts)


# Example usage:
tokenizer = tiktoken.get_encoding("gpt2")

train_dataset = AIDetectionDataset("train.csv", tokenizer=tokenizer)
val_dataset = AIDetectionDataset("validation.csv", tokenizer=tokenizer)
test_dataset = AIDetectionDataset("test.csv", tokenizer=tokenizer)

print(f"Max token length (train set): {train_dataset.max_length}")
print(f"Train samples: {len(train_dataset)} | Validation samples: {len(val_dataset)}")

# Example: get one batch item
sample_input, sample_label = train_dataset[0]
print("Sample token IDs:", sample_input[:20])
print("Sample label:", sample_label)


Max token length (train set): 1024
Train samples: 215 | Validation samples: 54
Sample token IDs: tensor([ 9704, 13629,   287,   262,  9552,   995, 22137,   287,   262,  9552,
          995,   318,   257,   845,  3024,  7243,   287,   284, 12545,   640])
Sample label: tensor(0)


In [7]:
from torch.utils.data import DataLoader
import torch

num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False,
)

print(f"Train batches: {len(train_loader)} | Val batches: {len(val_loader)} | Test batches: {len(test_loader)}")


Train batches: 26 | Val batches: 7 | Test batches: 33


In [8]:
print("Train loader:")
for input_batch, target_batch in train_loader:
    pass

print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions", target_batch.shape)

Train loader:
Input batch dimensions: torch.Size([8, 1024])
Label batch dimensions torch.Size([8])


In [9]:
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

assert train_dataset.max_length <= BASE_CONFIG["context_length"], (
    f"Dataset length {train_dataset.max_length} exceeds model's context "
    f"length {BASE_CONFIG['context_length']}. Reinitialize data sets with "
    f"`max_length={BASE_CONFIG['context_length']}`"
)

In [10]:
import os
import urllib.request

# import requests
import json
import numpy as np
import tensorflow as tf
from tqdm import tqdm


def download_and_load_gpt2(model_size, models_dir):
    # Validate model size
    allowed_sizes = ("124M", "355M", "774M", "1558M")
    if model_size not in allowed_sizes:
        raise ValueError(f"Model size not in {allowed_sizes}")

    # Define paths
    model_dir = os.path.join(models_dir, model_size)
    base_url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
    backup_base_url = "https://f001.backblazeb2.com/file/LLMs-from-scratch/gpt2"
    filenames = [
        "checkpoint", "encoder.json", "hparams.json",
        "model.ckpt.data-00000-of-00001", "model.ckpt.index",
        "model.ckpt.meta", "vocab.bpe"
    ]

    # Download files
    os.makedirs(model_dir, exist_ok=True)
    for filename in filenames:
        file_url = os.path.join(base_url, model_size, filename)
        backup_url = os.path.join(backup_base_url, model_size, filename)
        file_path = os.path.join(model_dir, filename)
        download_file(file_url, file_path, backup_url)

    # Load settings and params
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
    settings = json.load(open(os.path.join(model_dir, "hparams.json"), "r", encoding="utf-8"))
    params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)

    return settings, params


def download_file(url, destination, backup_url=None):
    def _attempt_download(download_url):
        with urllib.request.urlopen(download_url) as response:
            # Get the total file size from headers, defaulting to 0 if not present
            file_size = int(response.headers.get("Content-Length", 0))

            # Check if file exists and has the same size
            if os.path.exists(destination):
                file_size_local = os.path.getsize(destination)
                if file_size == file_size_local:
                    print(f"File already exists and is up-to-date: {destination}")
                    return True  # Indicate success without re-downloading

            block_size = 1024  # 1 Kilobyte

            # Initialize the progress bar with total file size
            progress_bar_description = os.path.basename(download_url)
            with tqdm(total=file_size, unit="iB", unit_scale=True, desc=progress_bar_description) as progress_bar:
                with open(destination, "wb") as file:
                    while True:
                        chunk = response.read(block_size)
                        if not chunk:
                            break
                        file.write(chunk)
                        progress_bar.update(len(chunk))
            return True

    try:
        if _attempt_download(url):
            return
    except (urllib.error.HTTPError, urllib.error.URLError):
        if backup_url is not None:
            print(f"Primary URL ({url}) failed. Attempting backup URL: {backup_url}")
            try:
                if _attempt_download(backup_url):
                    return
            except urllib.error.HTTPError:
                pass

        # If we reach here, both attempts have failed
        error_message = (
            f"Failed to download from both primary URL ({url})"
            f"{' and backup URL (' + backup_url + ')' if backup_url else ''}."
            "\nCheck your internet connection or the file availability.\n"
            "For help, visit: https://github.com/rasbt/LLMs-from-scratch/discussions/273"
        )
        print(error_message)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Alternative way using `requests`
"""
def download_file(url, destination):
    # Send a GET request to download the file in streaming mode
    response = requests.get(url, stream=True)

    # Get the total file size from headers, defaulting to 0 if not present
    file_size = int(response.headers.get("content-length", 0))

    # Check if file exists and has the same size
    if os.path.exists(destination):
        file_size_local = os.path.getsize(destination)
        if file_size == file_size_local:
            print(f"File already exists and is up-to-date: {destination}")
            return

    # Define the block size for reading the file
    block_size = 1024  # 1 Kilobyte

    # Initialize the progress bar with total file size
    progress_bar_description = url.split("/")[-1]  # Extract filename from URL
    with tqdm(total=file_size, unit="iB", unit_scale=True, desc=progress_bar_description) as progress_bar:
        # Open the destination file in binary write mode
        with open(destination, "wb") as file:
            # Iterate over the file data in chunks
            for chunk in response.iter_content(block_size):
                progress_bar.update(len(chunk))  # Update progress bar
                file.write(chunk)  # Write the chunk to the file
"""


def load_gpt2_params_from_tf_ckpt(ckpt_path, settings):
    # Initialize parameters dictionary with empty blocks for each layer
    params = {"blocks": [{} for _ in range(settings["n_layer"])]}

    # Iterate over each variable in the checkpoint
    for name, _ in tf.train.list_variables(ckpt_path):
        # Load the variable and remove singleton dimensions
        variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))

        # Process the variable name to extract relevant parts
        variable_name_parts = name.split("/")[1:]  # Skip the 'model/' prefix

        # Identify the target dictionary for the variable
        target_dict = params
        if variable_name_parts[0].startswith("h"):
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]

        # Recursively access or create nested dictionaries
        for key in variable_name_parts[1:-1]:
            target_dict = target_dict.setdefault(key, {})

        # Assign the variable array to the last key
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array

    return params

2025-10-10 06:52:10.434297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760079130.620192      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760079130.674622      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
import numpy as np
import tiktoken
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#####################################
# Chapter 2
#####################################


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


#####################################
# Chapter 3
#####################################
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec


#####################################
# Chapter 4
#####################################
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_resid = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_resid(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_resid(x)
        x = x + shortcut  # Add the original input back

        return x


class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_token, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Get the idx of the vocab entry with the highest logits value
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


#####################################
# Chapter 5
#####################################
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))


def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])

    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(
            gpt.trf_blocks[b].att.W_value.bias, v_b)

        gpt.trf_blocks[b].att.out_proj.weight = assign(
            gpt.trf_blocks[b].att.out_proj.weight,
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign(
            gpt.trf_blocks[b].att.out_proj.bias,
            params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight,
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias,
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight,
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias,
            params["blocks"][b]["mlp"]["c_proj"]["b"])

        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale,
            params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift,
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale,
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift,
            params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])


def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # add batch dimension
    return encoded_tensor


def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)  # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [12]:
model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir="gpt2")

model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval();

checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 156kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 5.10MiB/s]
hparams.json: 100%|██████████| 90.0/90.0 [00:00<00:00, 177kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [01:02<00:00, 8.00MiB/s]
model.ckpt.index: 100%|██████████| 5.21k/5.21k [00:00<00:00, 9.24MiB/s]
model.ckpt.meta: 100%|██████████| 471k/471k [00:00<00:00, 3.15MiB/s]
vocab.bpe: 100%|██████████| 456k/456k [00:00<00:00, 4.13MiB/s]


In [13]:
text_1 = "Every effort moves you"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_1, tokenizer),
    max_new_tokens=15,
    context_size=BASE_CONFIG["context_length"]
)

print(token_ids_to_text(token_ids, tokenizer))

Every effort moves you forward.

The first step is to understand the importance of your work


In [14]:
text_2 = (
    "Is the following text 'spam'? Answer with 'yes' or 'no':"
    " 'You are a winner you have been specially"
    " selected to receive $1000 cash or a $2000 award.'"
)

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_2, tokenizer),
    max_new_tokens=23,
    context_size=BASE_CONFIG["context_length"]
)

print(token_ids_to_text(token_ids, tokenizer))

Is the following text 'spam'? Answer with 'yes' or 'no': 'You are a winner you have been specially selected to receive $1000 cash or a $2000 award.'

The following text 'spam'? Answer with 'yes' or 'no': 'You are a winner


In [15]:
print(model)

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_resid): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768,

In [16]:
for param in model.parameters():
    param.requires_grad = False

In [17]:
torch.manual_seed(123)

num_classes = 2
model.out_head = torch.nn.Linear(in_features=BASE_CONFIG["emb_dim"], out_features=num_classes)

In [18]:


# Unfreeze the last 3 transformer blocks
for block in model.trf_blocks[-4:]:
    for param in block.parameters():
        param.requires_grad = True

# Unfreeze the final normalization layer
for param in model.final_norm.parameters():
    param.requires_grad = True


In [19]:
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions:", inputs.shape) # shape: (batch_size, num_tokens)

Inputs: tensor([[5211,  345,  423,  640]])
Inputs dimensions: torch.Size([1, 4])


In [20]:
with torch.no_grad():
    outputs = model(inputs)

print("Outputs:\n", outputs)
print("Outputs dimensions:", outputs.shape) # shape: (batch_size, num_tokens, num_classes)

Outputs:
 tensor([[[-1.5854,  0.9904],
         [-3.7235,  7.4548],
         [-2.2661,  6.6049],
         [-3.5983,  3.9902]]])
Outputs dimensions: torch.Size([1, 4, 2])


In [21]:
print("Last output token:", outputs[:, -1, :])

Last output token: tensor([[-3.5983,  3.9902]])


In [22]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# --- Optimizer ---
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)

num_epochs = 20 # you can increase later
best_auc = 0.0

for epoch in range(num_epochs):
    print(f"\n===== EPOCH {epoch+1}/{num_epochs} =====")
    model.train()
    train_losses, train_probs, train_labels = [], [], []

    # --- Training ---
    for inputs, labels in tqdm(train_loader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)                   # (batch, seq_len, 2)
        logits = outputs[:, -1, :]                # last token
        loss = F.cross_entropy(logits, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track loss and predictions
        probs = F.softmax(logits, dim=1)[:, 1].detach().cpu().numpy()
        train_probs.extend(probs)
        train_labels.extend(labels.cpu().numpy())
        train_losses.append(loss.item())

    # Compute ROC-AUC for training
    train_auc = roc_auc_score(train_labels, train_probs)
    print(f"Train Loss: {sum(train_losses)/len(train_losses):.4f} | Train ROC-AUC: {train_auc:.4f}")

    # --- Validation ---
    model.eval()
    val_losses, val_probs, val_labels = [], [], []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validating", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            logits = outputs[:, -1, :]

            loss = F.cross_entropy(logits, labels)
            probs = F.softmax(logits, dim=1)[:, 1].cpu().numpy()

            val_losses.append(loss.item())
            val_probs.extend(probs)
            val_labels.extend(labels.cpu().numpy())

    val_auc = roc_auc_score(val_labels, val_probs)
    val_loss = sum(val_losses) / len(val_losses)

    print(f"Validation Loss: {val_loss:.4f} | Validation ROC-AUC: {val_auc:.4f}")

    # --- Checkpoint if improved ---
    if val_auc > best_auc:
        best_auc = val_auc
        torch.save(model.state_dict(), f"best_gpt2_model_auc{val_auc:.4f}.pt")
        print(f"✅ Saved new best model with ROC-AUC {val_auc:.4f}")



===== EPOCH 1/20 =====


Train Loss: 1.1244 | Train ROC-AUC: 0.4637


Validation Loss: 0.6893 | Validation ROC-AUC: 0.4821
✅ Saved new best model with ROC-AUC 0.4821

===== EPOCH 2/20 =====


Train Loss: 0.6721 | Train ROC-AUC: 0.6569


Validation Loss: 0.6713 | Validation ROC-AUC: 0.7830
✅ Saved new best model with ROC-AUC 0.7830

===== EPOCH 3/20 =====


...

Validation Loss: 0.2489 | Validation ROC-AUC: 0.9904

===== EPOCH 19/20 =====


Train Loss: 0.0051 | Train ROC-AUC: 1.0000


Validation Loss: 0.1052 | Validation ROC-AUC: 0.9986

===== EPOCH 20/20 =====


Train Loss: 0.0024 | Train ROC-AUC: 1.0000


Validation Loss: 0.0737 | Validation ROC-AUC: 0.9973


In [23]:
import torch
import pandas as pd
import tiktoken
import torch.nn.functional as F
from tqdm import tqdm
import glob

# ==========================
# Load and prepare test data
# ==========================
test_df = pd.read_csv("/kaggle/input/mercor-ai-detection/test.csv")

# Store IDs separately
ids = test_df["id"].values

# Combine text columns
test_df["text"] = test_df["topic"].astype(str) + " " + test_df["answer"].astype(str)

# ==========================
# Tokenize test data
# ==========================
tokenizer = tiktoken.get_encoding("gpt2")
max_length = 1024
pad_token_id = 50256

encoded_texts = [
    tokenizer.encode(str(text)) for text in test_df["text"]
]

# Truncate and pad
encoded_texts = [
    (encoded[:max_length] + [pad_token_id] * max(0, max_length - len(encoded)))
    for encoded in encoded_texts
]

# Convert to tensor
test_tensors = torch.tensor(encoded_texts, dtype=torch.long)

# ==========================
# Predict using trained model
# ==========================
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load best available model automatically
model_paths = glob.glob("best_gpt2_model_auc*.pt")
if not model_paths:
    raise FileNotFoundError("No trained model found in the current directory!")
best_model_path = sorted(model_paths)[-1]
print(f"✅ Loading model: {best_model_path}")

model.load_state_dict(torch.load(best_model_path, map_location=device))
model.to(device)
model.eval()

pred_labels = []

with torch.no_grad():
    for i in tqdm(range(0, len(test_tensors), 8), desc="Predicting"):
        batch_inputs = test_tensors[i:i+8].to(device)
        outputs = model(batch_inputs)
        logits = outputs[:, -1, :]
        probs = F.softmax(logits, dim=1)[:, 1]
        preds = (probs > 0.5).long().cpu().numpy()
        pred_labels.extend(preds)

# ==========================
# Create submission
# ==========================
submission = pd.DataFrame({
    "id": ids,
    "is_cheating": pred_labels
})

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv generated successfully!")
display(submission.head())


✅ Loading model: best_gpt2_model_auc1.0000.pt


Predicting: 100%|██████████| 33/33 [00:22<00:00,  1.48it/s]

✅ submission.csv generated successfully!


,id,is_cheating
0,form_r_AAABmJpaBSZniRCnvHlBs66K,0
1,form_r_JiL6ylwnijP66wTCo52XSP,1
2,form_r_AAABmLN7KCDw89RJl3RORa5n,1
3,form_r_AAABmN2ij3UoaExF4jxFcZ_7,0
4,form_r_AAABmJYbGzEuL3gArMVHMZ1P,1
